In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import astropy.units as u
import astropy.constants as const
import numpy as np

from astropy.coordinates import Distance
import seaborn as sns
sns.set(style='white', font_scale=1.2)

In [2]:
dist = Distance(value=470*u.lightyear)
dist.distmod

<Quantity 5.79335991 mag>

In [3]:
mv = 2.8 * u.mag
Mv = mv - dist.distmod
Mv

<Quantity -2.99335991 mag>

In [4]:
Mbol = Mv - 3.16*u.mag
Mbol

<Quantity -6.15335991 mag>

In [5]:
L = 10**((Mbol.value - 4.74)/-2.5) * u.solLum
L

<Quantity 22769.00043849 solLum>

In [6]:
Teff = 3e4 * u.K
R = np.sqrt(L/(4*np.pi*const.sigma_sb * Teff**4)).to(u.solRad)
R

<Quantity 5.58576832 solRad>

In [7]:
M = (L.value / 1.5)**(1/3.5) * u.solMass
M

<Quantity 15.65465966 solMass>

In [8]:
g = const.G * M / R**2
g.cgs

<Quantity 13757.68689006 cm / s2>

In [9]:
np.log10(g.cgs.value)

4.138545421157457

In [10]:
vesc = np.sqrt(2*const.G*M/R)
vesc.cgs

<Quantity 1.03404674e+08 cm / s>

In [11]:
rho = M/(4/3 * np.pi * R**3)
rho.cgs

<Quantity 0.12663696 g / cm3>

In [13]:
g_sol = const.G * u.solMass / (u.solRad**2)
g.cgs / g_sol.cgs

<Quantity 0.50173892>

In [14]:
vesc_sol = np.sqrt(2*const.G*u.solMass/u.solRad)
vesc.cgs / vesc_sol.cgs

<Quantity 1.67409599>

In [15]:
rho_sol = u.solMass/(4/3 * np.pi * u.solRad**3)
rho.cgs / rho_sol.cgs

<Quantity 0.08982451>